##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Resposta a perguntas BERT com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/question_answer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/question_answer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

A [biblioteca TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) simplifica o processo de adaptar e converter um modelo do TensorFlow para dados de entrada específicos ao implantar esse modelo em aplicativos de aprendizado de máquina em dispositivos.

Este notebook apresenta um exemplo completo que utiliza a biblioteca Model Maker para ilustrar a adaptação e conversão de um modelo de resposta a perguntas usado com frequência para tarefas de resposta a perguntas.

# Introdução à tarefa de resposta a perguntas BERT

A tarefa com suporte nesta biblioteca é uma tarefa de extrair a resposta a uma pergunta: dado um trecho e uma pergunta, a resposta está presente no trecho. A imagem abaixo mostra um exemplo de resposta a uma pergunta.

<p align="center"><img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_squad_showcase.png" width="500"></p>

<p align="center">
    <em>As respostas estão presentes no trecho (crédito da imagem: <a href="https://rajpurkar.github.io/mlx/qa-and-squad/">blog SQuAD</a>)</em>
</p>

Quanto ao modelo de tarefa de resposta a perguntas, as entradas devem ser o par trecho/pergunta que já foram pré-processadas, e as saídas devem ser os logits de início e fim para cada token do trecho. O tamanho da entrada pode ser definido e ajustado de acordo com o tamanho do trecho e da pergunta.

## Visão geral completa


O trecho de código abaixo demonstra como obter o modelo com algumas linhas de código. O processo geral inclui 5 etapas: (1) escolher um modelo, (2) carregar dados, (3) retreinar o modelo, (4) avaliar e (5) exportá-lo para o formato do TensorFlow Lite.

```python
# Chooses a model specification that represents the model.
spec = model_spec.get('mobilebert_qa')

# Gets the training data and validation data.
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

# Fine-tunes the model.
model = question_answer.create(train_data, model_spec=spec)

# Gets the evaluation result.
metric = model.evaluate(validation_data)

# Exports the model to the TensorFlow Lite format with metadata in the export directory.
model.export(export_dir)
```

As próximas seções explicam o código com maiores detalhes.

## Pré-requisitos

Para executar este exemplo, instale os pacotes exigidos, incluindo o pacote do Model Maker no [repositório do GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker-nightly

Importe os pacotes necessários.

In [ ]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import question_answer
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.question_answer import DataLoader

A "Visão geral completa" demonstra um exemplo completo simples. A próxima seção mostra mais detalhes do exemplo.

## Escolha um model_spec que represente um modelo de resposta a perguntas

Cada objeto `model_spec` representa um modelo específico de resposta a perguntas. Atualmente, o Model Maker tem suporte a modelos MobileBERT e BERT-Base.

Modelo com suporte | Nome de model_spec | Descrição do modelo
--- | --- | ---
[MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa' | 4,3 vezes menor e 5,5 vezes mais rápido do que o BERT-Base, alcançando resultados competitivos, adequados para aplicativos em dispositivos.
[MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) | 'mobilebert_qa_squad' | Mesma arquitetura de modelo do MobileBERT, e o modelo inicial já é retreinado com [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/).
[BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) | 'bert_qa' | Modelo BERT padrão amplamente usado em tarefas de NLP.

Neste tutorial, [MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) é usado como exemplo. Como o modelo já foi retreinado com [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/), pode convergir mais rapidamente em tarefas de resposta a perguntas.


In [ ]:
spec = model_spec.get('mobilebert_qa_squad')

## Carregue dados de entrada específicos em um aplicativo de aprendizado de máquina em dispositivo e pré-processe os dados

[TriviaQA](https://nlp.cs.washington.edu/triviaqa/) é um dataset de compreensão de leitura que contém mais de 650 mil tuplas pergunta-resposta-evidência. Neste tutorial, você usará um subconjunto desse dataset para aprender a usar a biblioteca Model Maker.

Para carregar os dados, converta o dataset TriviaQA para o formato [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/) executando o [script conversor do Python](https://github.com/mandarjoshi90/triviaqa#miscellaneous) com `--sample_size=8000` e um conjunto de dados `web`. Modifique ligeiramente o código de conversão:

- Ignore as amostras que não conseguiram encontrar uma resposta no documento do contexto.
- Obtenha a resposta original no contexto sem diferenciar letras maiúsculas ou minúsculas.

Baixe a versão arquivada do dataset já convertido.

In [ ]:
train_data_path = tf.keras.utils.get_file(
    fname='triviaqa-web-train-8000.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-web-train-8000.json')
validation_data_path = tf.keras.utils.get_file(
    fname='triviaqa-verified-web-dev.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-verified-web-dev.json')

Você também pode treinar o modelo MobileBERT com seu próprio dataset. Se você estiver executando este notebook no Colab, carregue os dados pela barra lateral esquerda.


<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_question_answer.png" width="800" hspace="100" alt="Upload File">

Se você preferir não carregar os dados na nuvem, pode executar a biblioteca offline de acordo com este [guia](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

Use o método `DataLoader.from_squad` para carregar e pré-processar os dados no [formato SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) para um `model_spec` específico. Você pode usar o formato SQuAD2.0 ou SQuAD1.1. Ao definir o parâmetro `version_2_with_negative` como `True` (verdadeiro), o formato será SQuAD2.0. Caso contrário, será SQuAD1.1. Por padrão, `version_2_with_negative` é `False` (falso).

In [ ]:
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

## Personalize o modelo do TensorFlow

Crie um modelo personalizado de resposta a perguntas com base nos dados carregados. A função `create` consiste nas seguintes etapas:

1. Cria o modelo de resposta a perguntas de acordo com `model_spec`.
2. Treina o modelo de resposta a perguntas. As épocas padrão e o tamanho de lote padrão são definidos pelas duas variáveis `default_training_epochs` e `default_batch_size` no objeto `model_spec`.

In [ ]:
model = question_answer.create(train_data, model_spec=spec)

Confira a estrutura detalhada do modelo.

In [ ]:
model.summary()

## Avalie o modelo personalizado

Avalie o modelo e os dados de validação e obtenha um dicionário das métricas, incluindo a pontuação `f1` e `exact match`, etc. Observe que as métricas de SQuAD1.1 e SQuAD2.0 são diferentes.

In [ ]:
model.evaluate(validation_data)

## Exporte para um modelo do TensorFlow Lite

Converta o modelo treinado para o formato de modelos do TensorFlow Lite com [metadados](https://www.tensorflow.org/lite/models/convert/metadata) para poder usá-lo posteriormente em um aplicativo de aprendizado de máquina em dispositivos. O arquivo de vocabulário é incorporado aos metadados. O nome de arquivo padrão do TF Lite é `model.tflite`.

Em diversos aplicativos de aprendizado de máquina em dispositivos, o tamanho do modelo é um fator importante. Portanto, recomendamos aplicar quantização no modelo para deixá-lo menor e possivelmente mais rápido. Para modelos BERT e MobileBERT, a técnica padrão de quantização pós-treinamento é a quantização de intervalo dinâmico.

In [ ]:
model.export(export_dir='.')

Você pode usar o arquivo de modelo do TensorFlow Lite no aplicativo de referência [bert_qa](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) utilizando a [API BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) na [biblioteca Task do TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview), basta baixá-lo na barra lateral esquerda do Colab.

Confira abaixo os formatos de exportação permitidos:

- `ExportFormat.TFLITE`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

Por padrão, só é exportado o modelo do TensorFlow Lite com metadados. Você também pode exportar diferentes arquivos seletivamente. Por exemplo: exporte somente o arquivo de vocabulário da seguinte forma:

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.VOCAB)

E você pode avaliar o modelo do TF Lite com o método `evaluate_tflite`. É esperado que essa etapa demore um tempo longo.

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

## Uso avançado

A função `create` é uma parte essencial dessa biblioteca, em que o parâmetro `model_spec` define a especificação do modelo. Atualmente, há suporte à classe `BertQASpec`. Há dois modelos: MobileBERT e BERT-Base. A função `create` consiste nas seguintes etapas:

1. Cria o modelo de resposta a perguntas de acordo com `model_spec`.
2. Treina o modelo de resposta a perguntas.

Nesta seção, descreveremos diversos tópicos avançados, incluindo como alterar o modelo, ajustar os hiperparâmetros de treinamento, etc.

### Ajuste o modelo

Você pode ajustar a infraestrutura do modelo, como os parâmetros `seq_len` e `query_len`, na classe `BertQASpec`.

Parâmetros do modelo ajustáveis:

- `seq_len`: tamanho do trecho a ser alimentado no modelo.
- `query_len`: tamanho da pergunta a ser alimentada no modelo.
- `doc_stride`: stride ao usar a estratégia de janela deslizante para pegar partes do documento.
- `initializer_range`: desvio padrão do truncated_normal_initializer para inicializar as matrizes de pesos.
- `trainable`: booleano, indica se a camada pré-treinada é treinável.

Parâmetros do pipeline de treinamento ajustáveis:

- `model_dir`: local dos arquivos de checkpoint do modelo. Caso não seja definido, será usado um diretório temporário.
- `dropout_rate`: taxa de dropout.
- `learning_rate`: taxa de aprendizado inicial para Adam.
- `predict_batch_size`: tamanho do lote para previsão.
- `tpu`: endereço da TPU à qual se conectar. Usado somente ao utilizar TPU.


Por exemplo: você pode treinar o modelo com um tamanho de sequência maior. Se você alterar o modelo, primeiro precisa construir um novo `model_spec`.

In [ ]:
new_spec = model_spec.get('mobilebert_qa')
new_spec.seq_len = 512

As outras etapas são as mesmas. Observação: você precisa executar novamente tanto `dataloader` quanto `create`, pois especificações diferentes de modelos podem ter etapas de pré-processamento diferentes.


### Ajuste os hiperparâmetros de treinamento

Você também pode ajustar os hiperparâmetros de treinamento, como `epochs` e `batch_size`, o que impacta o desempenho do modelo. Por exemplo:

- `epochs`: mais épocas podem levar a um desempenho melhor, mas podem causar overfitting.
- `batch_size`: número de amostras a serem usadas em um passo de treinamento.

Por exemplo, você pode treinar com mais épocas e um tamanho de lote maior:

```python
model = question_answer.create(train_data, model_spec=spec, epochs=5, batch_size=64)
```

### Altere a arquitetura do modelo

É possível alterar o modelo base usado para treinar os dados mudando `model_spec`. Por exemplo, para alterar para o modelo BERT-Base, execute:

```python
spec = model_spec.get('bert_qa')
```

As outras etapas são as mesmas.

### Personalize a quantização pós-treinamento em um modelo do TensorFlow Lite

A [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization) é uma técnica de conversão que pode reduzir o tamanho do modelo e a latência de inferência, além de aumentar a velocidade de inferência da CPU e do acelerador de hardware com uma pequena redução da exatidão do modelo. A quantização é amplamente utilizada para otimizar o modelo.

A biblioteca Model Maker aplica uma técnica padrão de quantização pós-treinamento ao exportar o modelo. Se você quiser personalizar a quantização pós-treinamento, o Model Maker oferece suporte a diversas opções usando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Vejamos a quantização de float16 como exemplo. Primeiro, definimos a configuração de quantização.

```python
config = QuantizationConfig.for_float16()
```

Em seguida, exportamos o modelo do TensorFlow Lite com essa configuração.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Saiba mais

Leia o exemplo de [pergunta e resposta BERT](https://www.tensorflow.org/lite/examples/bert_qa/overview) para aprender os detalhes técnicos. Confira mais informações em:

- [Guia](https://www.tensorflow.org/lite/models/modify/model_maker) e [referência da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) do TensorFlow Lite Model Maker.
- Task Library: [BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) para implantação.
- Aplicativos de referência completos para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) e [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/ios).